In [ ]:
!pip install -q torch torchvision torchaudio transformers gradio openai

In [2]:
# Disable TensorFlow usage in Transformers
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# Imports
import gradio as gr
from transformers import pipeline
from openai import OpenAI

# Set or paste your OpenAI API key here, eg "sk-XXXXXXXXXXXXXXXXXXXXXXXX"
client = OpenAI(api_key="OPENAI_API_KEY")

In [3]:
# Load default sentiment analysis model (DistilBERT fine-tuned for sentiment)
sentiment_analyzer = pipeline("sentiment-analysis", framework="pt")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [4]:
def analyze_review(review_text):
    if not review_text.strip():
        return "⚠️ Please enter a review.", "", ""
    
    # 1️⃣ Sentiment Analysis
    sentiment_result = sentiment_analyzer(review_text)[0]
    label = sentiment_result['label']
    score = round(sentiment_result['score'] * 100, 2)
    sentiment_output = f"{label} ({score}%)"

    # 2️⃣ AI Explanation
    explanation_prompt = f"Explain in one sentence why this review is {label.lower()}: {review_text}"
    explanation = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": explanation_prompt}]
    )
    explanation_text = explanation.choices[0].message.content.strip()

    # 3️⃣ AI Rephrase
    rephrase_prompt = f"Rephrase this review in a neutral and brand-friendly tone: {review_text}"
    rephrase = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": rephrase_prompt}]
    )
    rephrase_text = rephrase.choices[0].message.content.strip()

    return sentiment_output, explanation_text, rephrase_text

In [5]:
with gr.Blocks() as demo:
    gr.Markdown("# 🛍 Product Review Sentiment + Reasoning")
    gr.Markdown("Analyze customer reviews, explain why they are positive/negative, and rephrase them.")

    review_input = gr.Textbox(lines=5, placeholder="Paste your product review here...", label="Customer Review")
    sentiment_output = gr.Label(label="Sentiment Result")
    explanation_output = gr.Textbox(label="AI Explanation", interactive=False)
    rephrase_output = gr.Textbox(label="Rephrased Review", interactive=False)

    analyze_button = gr.Button("Analyze Review")

    analyze_button.click(
        analyze_review,
        inputs=[review_input],
        outputs=[sentiment_output, explanation_output, rephrase_output]
    )

In [6]:
# Launch with share link (~72 hours)
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://4a0b95d94c15db35a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
